# One tailed t-test

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#data = pd.read_csv('machine.txt', sep = ' ', encoding = 'utf-8')
data = pd.read_csv('files_for_lab/machine.txt', encoding='UTF-16' , sep = '\t')

In [23]:
data

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


# Matched Pairs Test

In [29]:
pokemon = pd.read_csv('files_for_lab/pokemon.csv')

In [30]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


H0 : defense score = attack score
H1 : defense score != attack score
--> two-sided test

In [31]:
attack_vs_def = pokemon[['Attack', 'Defense']]
attack_vs_def

,Attack,Defense
0,49,49
1,62,63
2,82,83
3,100,123
4,52,43
...,...,...
795,100,150
796,160,110
797,110,60
798,160,60


In [32]:
attack_vs_def['difference'] = attack_vs_def['Attack'] - attack_vs_def['Defense']

/var/folders/tk/75w8q4_s16n1_zv4jdmdlwhh0000gn/T/ipykernel_43411/1352146092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attack_vs_def['difference'] = attack_vs_def['Attack'] - attack_vs_def['Defense']


In [33]:
sample = attack_vs_def.sample(30)
sample.head()

,Attack,Defense,difference
758,52,67,-15
133,50,35,15
679,66,84,-18
641,44,50,-6
295,70,70,0


In [34]:
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(11.866666666666667, 35.396018998180296)

In [35]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: 11.87
The standard deviation of our samples differences is: 35.40
Our t statistics is: 1.84


In [37]:
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703

The negative of our critical value is -2.05, and our t statistic is -1.84 --> -2.05 < -1.84 --> we reject H0.

Let's try with p-value now

In [43]:
st.t.cdf(t,df = len(sample['Attack']) + len(sample['Defense']) -2)

0.9642772074802455

In [41]:
st.ttest_rel(sample['Attack'],sample['Defense'])

Ttest_relResult(statistic=1.836263286010248, pvalue=0.07659039596140287)

p-value are different in both formulas... weird.
But in both cases, p-value > 0.025, so we reject H0

# ANOVA

STEPS : Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

In [48]:
data = pd.read_excel('files_for_lab/anova_lab_data.xlsx')

In [49]:
data.head()

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


In [58]:
data.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [50]:
data.shape

(15, 2)

In [52]:
# What are the degrees of freedom of model, error terms, and total DoF
degree_freedom = len(data) - 1
degree_freedom

14

In [60]:
st.f_oneway(data[data['Power '] == '160 W']['Etching Rate'],
            data[data['Power '] == '180 W']['Etching Rate'],
            data[data['Power '] == '200 W']['Etching Rate'])

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

p-value > 0.5, we accept H0

In [61]:
# F Table
S2t = 0
for power in data['Power '].unique():
    ng = len(data[data['Power '] == power])  
    S2t  += ( ( data[data['Power '] == power]['Etching Rate'].mean() - data['Etching Rate'].mean() ) ** 2) * ng
S2t /= ( data['Power '].nunique() - 1 )  
print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


In [62]:
S2E = 0
for power in data['Power '].unique():
    for rate in data[data['Power '] == power]['Etching Rate']:
        S2E += ( rate - data[data['Power '] == power]['Etching Rate'].mean() ) ** 2
S2E /= ( len(data) - data['Power '].nunique() ) 

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [63]:
# Finally we can compute F
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88
